If you are running this on Google Colab, you need to uncomment (remove the `#`) and execute the following lines to install the Pyomo package, the solver, and some helper tools. If you are running this on Binder or elsewhere (e.g. your own computer) you can ignore this.

In [1]:
# !pip install pyomo==6.4.1
# !apt install coinor-libipopt-dev
# !pip install ipopt
# !pip install "git+https://github.com/sjpfenninger/sen1511.git#egg=sen1511utils&subdirectory=sen1511utils"

In [2]:
import pyomo.environ as pyo

from sen1511utils import summarise_results

# Assignment 4 - Nonlinear programming (NLP)

## NUMBER.a)

In [3]:
m = pyo.ConcreteModel(name = "Economic dispatch with quadratic cost functions")
m.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

##
# 1. Decision variables
##

m.PG1 = pyo.Var(domain=pyo.NonNegativeReals)
m.PG2 = pyo.Var(domain=pyo.NonNegativeReals)
m.PG3 = pyo.Var(domain=pyo.NonNegativeReals)

##
# 2. Objective function
##

cost_0 = 10 +    1 * m.PG1 + 0.5 *  0.1 * m.PG1**2
cost_1 = 20 +  1.1 * m.PG2 + 0.5 * 0.05 * m.PG2**2
cost_2 = 20 + 1.25 * m.PG3 + 0.5 *  0.1 * m.PG3**2

m.cost = pyo.Objective(
    expr = cost_0 + cost_1 + cost_2,
    sense = pyo.minimize,
)

##
# 3. Constraints
##

# For the case of 80 MW demand (other cases: 5 MW, 30 MW)
m.demand = pyo.Constraint(expr = m.PG1 + m.PG2 + m.PG3 == 80)

m.pg1_max = pyo.Constraint(expr = m.PG1 <= 30)
m.pg2_max = pyo.Constraint(expr = m.PG2 <= 40)
m.pg3_max = pyo.Constraint(expr = m.PG3 <= 50)

# Solve the problem
solver = pyo.SolverFactory('ipopt')
solver.solve(m)

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 4, 'Number of variables': 3, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.14.9\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.10702204704284668}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [4]:
summarise_results(m)

<div class="alert alert-block alert-info">

💡 As usual, you could use Python to automate - and make it easier to extend this to a larger problem with many more power plants. For example, you might specify the cost function as an actual Python function and and sum it up with `sum()` in the objective:

```python
PG = [m.PG1, m.PG2, m.PG3]
C = [10, 20, 20]
a = [1, 1.1, 1.25]
b = [0.1, 0.05, 0.1]

def cost(i):
    return C[i] + a[i] * PG[i] + 0.5 * b[i] * PG[i]**2

m.cost = pyo.Objective(
    expr = sum(cost[i] for i in range(3)),
    sense = pyo.minimize,
)
```

</div>

## NUMBER.b)

In [5]:
m = pyo.ConcreteModel(name = "Economic dispatch with quadratic cost functions and elastic demand")
m.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

##
# 1. Decision variables
##

# Generation
m.PG1 = pyo.Var(domain=pyo.NonNegativeReals)
m.PG2 = pyo.Var(domain=pyo.NonNegativeReals)
m.PG3 = pyo.Var(domain=pyo.NonNegativeReals)

# Demand
m.PD1 = pyo.Var(domain=pyo.NonNegativeReals)
m.PD2 = pyo.Var(domain=pyo.NonNegativeReals)

##
# 2. Objective function
##

PG = [m.PG1, m.PG2, m.PG3]
C = [10, 20, 20]
a = [1, 1.1, 1.25]
b = [0.1, 0.05, 0.1]

def cost(i):
    return C[i] + a[i] * PG[i] + 0.5 * b[i] * PG[i]**2

m.cost = pyo.Objective(
    expr = cost(0) + cost(1) + cost(2),
    sense = pyo.minimize,
)

##
# 3. Constraints
##

# For the case of 80 MW demand (other cases: 5 MW, 30 MW)
m.demand = pyo.Constraint(expr = m.PG1 + m.PG2 + m.PG3 == 80)

m.pg1_max = pyo.Constraint(expr = m.PG1 <= 30)
m.pg2_max = pyo.Constraint(expr = m.PG2 <= 40)
m.pg3_max = pyo.Constraint(expr = m.PG3 <= 50)

# Solve the problem
solver = pyo.SolverFactory('ipopt')
solver.solve(m)

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 4, 'Number of variables': 3, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.14.9\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.02503204345703125}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}